In [1]:
import pandas as pd
import ast
import numpy as np
from ast import literal_eval
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import re
from datetime import datetime
from dateutil import parser

Abro el archivo de json

In [2]:
input_file = 'australian_user_reviews.json'

# Leer el contenido del archivo JSON
with open(input_file, 'r', encoding='utf-8') as f:
    data = f.readlines()

# Convertir las líneas a registros JSON
records = [eval(line.strip()) for line in data]

# Crear el DataFrame a partir de los registros
df_reviews= pd.DataFrame(records)

In [3]:
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


Verificamos la estructura y tipo de dato del dataframe

In [4]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


desanidamos la columna review

In [5]:
unpacked_data = []
for index, row in df_reviews.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    for review_dict in row['reviews']:
        review_dict['user_id'] = user_id
        review_dict['user_url'] = user_url
        unpacked_data.append(review_dict)

df_reviews = pd.DataFrame(unpacked_data)

In [6]:
df_reviews

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637
...,...,...,...,...,...,...,...,...,...
59300,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,76561198312638244,http://steamcommunity.com/profiles/76561198312...
59301,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,76561198312638244,http://steamcommunity.com/profiles/76561198312...
59302,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,LydiaMorley,http://steamcommunity.com/id/LydiaMorley
59303,,Posted July 20.,,730,No ratings yet,True,:D,LydiaMorley,http://steamcommunity.com/id/LydiaMorley


verificamos duplicados 

In [7]:

duplicados = df_reviews[df_reviews.duplicated()]

cantidad_duplicados = len(duplicados)

if cantidad_duplicados == 0:
    print("No se encontraron filas duplicadas.")
else:
    print(f"Se encontraron {cantidad_duplicados} filas duplicadas:")
    print(duplicados)

Se encontraron 874 filas duplicadas:
      funny                      posted                    last_edited  \
1112         Posted September 24, 2015.                                  
2891           Posted January 10, 2014.                                  
2892           Posted January 10, 2014.                                  
2893          Posted December 17, 2014.                                  
2894           Posted January 13, 2014.                                  
...     ...                         ...                            ...   
44433                    Posted July 3.                                  
44434                    Posted June 1.                                  
44435           Posted August 17, 2014.                                  
44436         Posted February 17, 2014.                                  
44580             Posted July 18, 2015.  Last edited December 1, 2015.   

      item_id                                         helpful  recommend  

Eliminamos filas duplicadas 

In [8]:
# Eliminar filas duplicadas
df_reviews1 = df_reviews.drop_duplicates()

print("DataFrame original:")
print(df_reviews)
print("\nDataFrame sin filas duplicadas:")
print(df_reviews)

DataFrame original:
                                  funny                     posted  \
0                                         Posted November 5, 2011.   
1                                            Posted July 15, 2011.   
2                                           Posted April 21, 2011.   
3                                            Posted June 24, 2014.   
4                                        Posted September 8, 2013.   
...                                 ...                        ...   
59300                                              Posted July 10.   
59301                                               Posted July 8.   
59302  1 person found this review funny             Posted July 3.   
59303                                              Posted July 20.   
59304                                               Posted July 2.   

      last_edited item_id                                          helpful  \
0                    1250                                   N

Revisar el tipo de datos contenido dentro de cada columna

In [9]:
df_reviews = df_reviews1

In [10]:
def verificar_tipo_datos(df_reviews):
    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": []}
    for columna in df_reviews.columns:
        porcentaje_no_nulos = (df_reviews[columna].count() / len(df_reviews)) * 100
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df_reviews[columna].apply(type).unique())
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100-porcentaje_no_nulos, 2))
    df_info = pd.DataFrame(mi_dict)
    for columna in df_reviews1.columns:
        print(columna, " (nulos) = ", df_reviews[columna].isnull().sum())
    print("\nfilas completamente nulas: ", df_reviews.isna().all(axis=1).sum())
    return df_info 
df_info = verificar_tipo_datos(df_reviews)
print(df_info)

funny  (nulos) =  0
posted  (nulos) =  0
last_edited  (nulos) =  0
item_id  (nulos) =  0
helpful  (nulos) =  0
recommend  (nulos) =  0
review  (nulos) =  0
user_id  (nulos) =  0
user_url  (nulos) =  0

filas completamente nulas:  0
  nombre_campo        tipo_datos  no_nulos_%  nulos_%
0        funny   [<class 'str'>]       100.0      0.0
1       posted   [<class 'str'>]       100.0      0.0
2  last_edited   [<class 'str'>]       100.0      0.0
3      item_id   [<class 'str'>]       100.0      0.0
4      helpful   [<class 'str'>]       100.0      0.0
5    recommend  [<class 'bool'>]       100.0      0.0
6       review   [<class 'str'>]       100.0      0.0
7      user_id   [<class 'str'>]       100.0      0.0
8     user_url   [<class 'str'>]       100.0      0.0


In [11]:
df_reviews

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637
...,...,...,...,...,...,...,...,...,...
59300,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,76561198312638244,http://steamcommunity.com/profiles/76561198312...
59301,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,76561198312638244,http://steamcommunity.com/profiles/76561198312...
59302,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,LydiaMorley,http://steamcommunity.com/id/LydiaMorley
59303,,Posted July 20.,,730,No ratings yet,True,:D,LydiaMorley,http://steamcommunity.com/id/LydiaMorley


Elimino columnas no necesarias para optimizar el trabajo

In [12]:
columnas_a_eliminar = ['funny', 'last_edited']
df = df_reviews.drop(columns=columnas_a_eliminar)

In [13]:
df_review = df

In [14]:
df_review

,posted,item_id,helpful,recommend,review,user_id,user_url
0,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,"Posted July 15, 2011.",22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637
4,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637
...,...,...,...,...,...,...,...
59300,Posted July 10.,70,No ratings yet,True,a must have classic from steam definitely wort...,76561198312638244,http://steamcommunity.com/profiles/76561198312...
59301,Posted July 8.,362890,No ratings yet,True,this game is a perfect remake of the original ...,76561198312638244,http://steamcommunity.com/profiles/76561198312...
59302,Posted July 3.,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,LydiaMorley,http://steamcommunity.com/id/LydiaMorley
59303,Posted July 20.,730,No ratings yet,True,:D,LydiaMorley,http://steamcommunity.com/id/LydiaMorley


se crea una columna llamada 'sentiment_analysis' que contiene etiquetas numéricas que representan el sentimiento asociado con cada revisión de texto en la columna 'review'. Estas etiquetas son 0 para "Malo", 2 para "Positivo" y 1 para "Neutral".

In [15]:
def get_sentiment_label(text):
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity
    
    if polarity < -0.2:
        return 0  # Malo
    elif polarity > 0.2:
        return 2  # Positivo
    else:
        return 1  # Neutral

# Aplicar la función a la columna 'review' y crear la nueva columna 'sentiment_analysis'
df_review['sentiment_analysis'] = df_review['review'].apply(get_sentiment_label)

# Reemplazar la columna 'review' con la nueva columna 'sentiment_analysis'
df_review.drop(columns=['review'], inplace=True)

# Mostrar las primeras filas para verificar los cambios
df_review.head()

,posted,item_id,helpful,recommend,user_id,user_url,sentiment_analysis
0,"Posted November 5, 2011.",1250,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1
1,"Posted July 15, 2011.",22200,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
2,"Posted April 21, 2011.",43110,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1
3,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,js41637,http://steamcommunity.com/id/js41637,1
4,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,js41637,http://steamcommunity.com/id/js41637,1


In [16]:
df_review

,posted,item_id,helpful,recommend,user_id,user_url,sentiment_analysis
0,"Posted November 5, 2011.",1250,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1
1,"Posted July 15, 2011.",22200,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
2,"Posted April 21, 2011.",43110,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1
3,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,js41637,http://steamcommunity.com/id/js41637,1
4,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,js41637,http://steamcommunity.com/id/js41637,1
...,...,...,...,...,...,...,...
59300,Posted July 10.,70,No ratings yet,True,76561198312638244,http://steamcommunity.com/profiles/76561198312...,2
59301,Posted July 8.,362890,No ratings yet,True,76561198312638244,http://steamcommunity.com/profiles/76561198312...,1
59302,Posted July 3.,273110,1 of 2 people (50%) found this review helpful,True,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1
59303,Posted July 20.,730,No ratings yet,True,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,2


eliminamos punto y coma de la columna posted 

In [17]:
df_review['posted'] = df_review['posted'].str.replace(',','')

In [18]:
df_review['posted'] = df_review['posted'].str.replace('.','')

In [19]:
df_review

,posted,item_id,helpful,recommend,user_id,user_url,sentiment_analysis
0,Posted November 5 2011,1250,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1
1,Posted July 15 2011,22200,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
2,Posted April 21 2011,43110,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1
3,Posted June 24 2014,251610,15 of 20 people (75%) found this review helpful,True,js41637,http://steamcommunity.com/id/js41637,1
4,Posted September 8 2013,227300,0 of 1 people (0%) found this review helpful,True,js41637,http://steamcommunity.com/id/js41637,1
...,...,...,...,...,...,...,...
59300,Posted July 10,70,No ratings yet,True,76561198312638244,http://steamcommunity.com/profiles/76561198312...,2
59301,Posted July 8,362890,No ratings yet,True,76561198312638244,http://steamcommunity.com/profiles/76561198312...,1
59302,Posted July 3,273110,1 of 2 people (50%) found this review helpful,True,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1
59303,Posted July 20,730,No ratings yet,True,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,2


In [20]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58431 entries, 0 to 59304
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   posted              58431 non-null  object
 1   item_id             58431 non-null  object
 2   helpful             58431 non-null  object
 3   recommend           58431 non-null  bool  
 4   user_id             58431 non-null  object
 5   user_url            58431 non-null  object
 6   sentiment_analysis  58431 non-null  int64 
dtypes: bool(1), int64(1), object(5)
memory usage: 3.2+ MB


elimino la palabra posted de la columna posted 

In [21]:
import pandas as pd


df_review['posted'] = df_review['posted'].str.replace('Posted', '')



In [22]:
df_review

,posted,item_id,helpful,recommend,user_id,user_url,sentiment_analysis
0,November 5 2011,1250,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1
1,July 15 2011,22200,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
2,April 21 2011,43110,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1
3,June 24 2014,251610,15 of 20 people (75%) found this review helpful,True,js41637,http://steamcommunity.com/id/js41637,1
4,September 8 2013,227300,0 of 1 people (0%) found this review helpful,True,js41637,http://steamcommunity.com/id/js41637,1
...,...,...,...,...,...,...,...
59300,July 10,70,No ratings yet,True,76561198312638244,http://steamcommunity.com/profiles/76561198312...,2
59301,July 8,362890,No ratings yet,True,76561198312638244,http://steamcommunity.com/profiles/76561198312...,1
59302,July 3,273110,1 of 2 people (50%) found this review helpful,True,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1
59303,July 20,730,No ratings yet,True,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,2


cambiamos el formao de la columna posted y lo convertimos a fecha

In [23]:
df_review['posted'] = df_review['posted'].astype(str)


In [24]:
import pandas as pd

# Supongamos que tienes un DataFrame llamado 'review_desanidado_date' con una columna 'posted'
# que contiene fechas en el formato 'November 5 2011'

# Asegúrate de que la columna 'posted' sea de tipo string
df_review['posted'] = df_review['posted'].astype(str)

# Elimina espacios adicionales al principio y al final de cada fecha
df_review['posted'] = df_review['posted'].str.strip()

# Intenta convertir las fechas al formato datetime, manejando los errores como valores nulos
df_review['posted'] = pd.to_datetime(df_review['posted'], errors='coerce')

# Filtra las filas con fechas no nulas
df_review = df_review.dropna(subset=['posted'])

# Luego, convierte las fechas al formato deseado (año/mes/día) como cadenas de texto
df_review['posted'] = df_review['posted'].dt.strftime('%Y/%m/%d')

# Ahora, la columna 'posted' contiene las fechas en formato de cadena de texto (año/mes/día)

C:\Users\TOCALACTEOS\AppData\Local\Temp\ipykernel_2260\370484602.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review['posted'] = df_review['posted'].dt.strftime('%Y/%m/%d')


In [25]:
df_review

,posted,item_id,helpful,recommend,user_id,user_url,sentiment_analysis
0,2011/11/05,1250,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1
1,2011/07/15,22200,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
2,2011/04/21,43110,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1
3,2014/06/24,251610,15 of 20 people (75%) found this review helpful,True,js41637,http://steamcommunity.com/id/js41637,1
4,2013/09/08,227300,0 of 1 people (0%) found this review helpful,True,js41637,http://steamcommunity.com/id/js41637,1
...,...,...,...,...,...,...,...
59252,2015/10/14,730,1 of 1 people (100%) found this review helpful,True,wayfeng,http://steamcommunity.com/id/wayfeng,1
59255,2015/10/10,253980,No ratings yet,True,76561198251004808,http://steamcommunity.com/profiles/76561198251...,2
59265,2015/10/31,730,No ratings yet,True,72947282842,http://steamcommunity.com/id/72947282842,0
59267,2015/12/14,730,No ratings yet,True,ApxLGhost,http://steamcommunity.com/id/ApxLGhost,1


In [26]:
df_review['posted'] = pd.to_datetime(df_review['posted'], format='%Y/%m/%d')

C:\Users\TOCALACTEOS\AppData\Local\Temp\ipykernel_2260\363745329.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review['posted'] = pd.to_datetime(df_review['posted'], format='%Y/%m/%d')


In [27]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48498 entries, 0 to 59276
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   posted              48498 non-null  datetime64[ns]
 1   item_id             48498 non-null  object        
 2   helpful             48498 non-null  object        
 3   recommend           48498 non-null  bool          
 4   user_id             48498 non-null  object        
 5   user_url            48498 non-null  object        
 6   sentiment_analysis  48498 non-null  int64         
dtypes: bool(1), datetime64[ns](1), int64(1), object(4)
memory usage: 2.6+ MB


In [28]:
df_review

,posted,item_id,helpful,recommend,user_id,user_url,sentiment_analysis
0,2011-11-05,1250,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1
1,2011-07-15,22200,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
2,2011-04-21,43110,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1
3,2014-06-24,251610,15 of 20 people (75%) found this review helpful,True,js41637,http://steamcommunity.com/id/js41637,1
4,2013-09-08,227300,0 of 1 people (0%) found this review helpful,True,js41637,http://steamcommunity.com/id/js41637,1
...,...,...,...,...,...,...,...
59252,2015-10-14,730,1 of 1 people (100%) found this review helpful,True,wayfeng,http://steamcommunity.com/id/wayfeng,1
59255,2015-10-10,253980,No ratings yet,True,76561198251004808,http://steamcommunity.com/profiles/76561198251...,2
59265,2015-10-31,730,No ratings yet,True,72947282842,http://steamcommunity.com/id/72947282842,0
59267,2015-12-14,730,No ratings yet,True,ApxLGhost,http://steamcommunity.com/id/ApxLGhost,1


cambiamos el formato de la columna recommend para mejor trato con funcion api

In [29]:
import pandas as pd

# Supongamos que tienes un DataFrame llamado df_review y deseas convertir la columna 'recommend' a tipo float

# Define una función que toma un valor y lo convierte en float
def formato_deseado(valor):
    try:
        return float(valor)
    except ValueError:
        return None  # Opcional: si el valor no es convertible, puedes manejarlo de alguna manera

# Aplica la función formato_deseado a la columna 'recommend' usando apply
df_review['recommend'] = df_review['recommend'].apply(formato_deseado)



C:\Users\TOCALACTEOS\AppData\Local\Temp\ipykernel_2260\3453500647.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review['recommend'] = df_review['recommend'].apply(formato_deseado)


In [30]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48498 entries, 0 to 59276
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   posted              48498 non-null  datetime64[ns]
 1   item_id             48498 non-null  object        
 2   helpful             48498 non-null  object        
 3   recommend           48498 non-null  float64       
 4   user_id             48498 non-null  object        
 5   user_url            48498 non-null  object        
 6   sentiment_analysis  48498 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 3.0+ MB


In [31]:
df_review

,posted,item_id,helpful,recommend,user_id,user_url,sentiment_analysis
0,2011-11-05,1250,No ratings yet,1.0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1
1,2011-07-15,22200,No ratings yet,1.0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
2,2011-04-21,43110,No ratings yet,1.0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1
3,2014-06-24,251610,15 of 20 people (75%) found this review helpful,1.0,js41637,http://steamcommunity.com/id/js41637,1
4,2013-09-08,227300,0 of 1 people (0%) found this review helpful,1.0,js41637,http://steamcommunity.com/id/js41637,1
...,...,...,...,...,...,...,...
59252,2015-10-14,730,1 of 1 people (100%) found this review helpful,1.0,wayfeng,http://steamcommunity.com/id/wayfeng,1
59255,2015-10-10,253980,No ratings yet,1.0,76561198251004808,http://steamcommunity.com/profiles/76561198251...,2
59265,2015-10-31,730,No ratings yet,1.0,72947282842,http://steamcommunity.com/id/72947282842,0
59267,2015-12-14,730,No ratings yet,1.0,ApxLGhost,http://steamcommunity.com/id/ApxLGhost,1


guardamos el archivo en csv

In [32]:
df_review.to_csv('df_reviewfinal.csv', index=False)


lo abrimos para comprobar

In [33]:
df_review= pd.read_csv(r"df_reviewfinal.csv")

In [34]:
df_review

,posted,item_id,helpful,recommend,user_id,user_url,sentiment_analysis
0,2011-11-05,1250,No ratings yet,1.0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1
1,2011-07-15,22200,No ratings yet,1.0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
2,2011-04-21,43110,No ratings yet,1.0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1
3,2014-06-24,251610,15 of 20 people (75%) found this review helpful,1.0,js41637,http://steamcommunity.com/id/js41637,1
4,2013-09-08,227300,0 of 1 people (0%) found this review helpful,1.0,js41637,http://steamcommunity.com/id/js41637,1
...,...,...,...,...,...,...,...
48493,2015-10-14,730,1 of 1 people (100%) found this review helpful,1.0,wayfeng,http://steamcommunity.com/id/wayfeng,1
48494,2015-10-10,253980,No ratings yet,1.0,76561198251004808,http://steamcommunity.com/profiles/76561198251...,2
48495,2015-10-31,730,No ratings yet,1.0,72947282842,http://steamcommunity.com/id/72947282842,0
48496,2015-12-14,730,No ratings yet,1.0,ApxLGhost,http://steamcommunity.com/id/ApxLGhost,1


In [35]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48498 entries, 0 to 48497
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   posted              48498 non-null  object 
 1   item_id             48498 non-null  int64  
 2   helpful             48498 non-null  object 
 3   recommend           48498 non-null  float64
 4   user_id             48498 non-null  object 
 5   user_url            48498 non-null  object 
 6   sentiment_analysis  48498 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 2.6+ MB
